# __CUDA__ __C__  _#_

<a href="http://docs.nvidia.com/cuda"><img src="imagen/CUDA.png" width="30%" /></a>

## Ejemplo básico: Suma de vectores.

>Recordemos como se ve un programa de C. Posteriormente el entorno puro de CUDA C.
Como veremos a continuacion realmente programar la targeta grafica se reduce en primera instancia a tener un codigo en C# y agregar sobre este funciones de propias de CUDA (o *kernels*). Estas funciones o _kernels_ son ejecutados en la GPU de forma paralela.

>En este primer ejemplo es evidente la manera de paralelizar la suma de vectores
![Alt text](imagen/suma.png)

### Version C

```c
#include <stdio.h>

int main(void)
{
int N = 10;
float a[N],b[N],c[N];

for (int i = 0; i < N; ++i){
	a[i] = i;
	b[i] = 2.0f;	
}

for (int i = 0; i < N; ++i){
	c[i]= a[i]+b[i];	
}

for (int i = 0; i < N; ++i){
	printf("%f \n",c[i]);	
}


return 0;
}```

In [1]:
!g++ cpuAdd.c -o cpua

In [2]:
!cat cpuAdd.c

#include <stdio.h>

int main(void)
{
int N = 10;
float a[N],b[N],c[N];

for (int i = 0; i < N; ++i){
	a[i] = i;
	b[i] = 2.0f;	
}

for (int i = 0; i < N; ++i){
	c[i]= a[i]+b[i];	
}

for (int i = 0; i < N; ++i){
	printf("%f \n",c[i]);	
}


return 0;
}


In [3]:
!./cpua

2.000000 
3.000000 
4.000000 
5.000000 
6.000000 
7.000000 
8.000000 
9.000000 
10.000000 
11.000000 


### Version CUDA C

![Alt text](imagen/cuda3.png)

![Alt text](imagen/CUDAmodelThreads.png)

```c
#include <stdio.h>
#include <cuda_runtime.h>
// CUDA Kernel
__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

/**
 * Host main routine
 */
int main(void)
{
    int numElements = 15;
    size_t size = numElements * sizeof(float);
    printf("[Vector addition of %d elements]\n", numElements);

    float a[numElements],b[numElements],c[numElements];
    float *a_gpu,*b_gpu,*c_gpu;

    cudaMalloc((void **)&a_gpu, size);
    cudaMalloc((void **)&b_gpu, size);
    cudaMalloc((void **)&c_gpu, size);

    for (int i=0;i<numElements;++i ){
    
    	a[i] = i*i;
    	b[i] = i;
    
    }
    // Copy the host input vectors A and B in host memory to the device input vectors in
    // device memory
    printf("Copy input data from the host memory to the CUDA device\n");
    cudaMemcpy(a_gpu, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b_gpu, b, size, cudaMemcpyHostToDevice);

    // Launch the Vector Add CUDA Kernel
    int threadsPerBlock = 256;
    int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(a_gpu, b_gpu, c_gpu, numElements);

    // Copy the device result vector in device memory to the host result vector
    // in host memory.
    printf("Copy output data from the CUDA device to the host memory\n");
    cudaMemcpy(c, c_gpu, size, cudaMemcpyDeviceToHost);

    for (int i=0;i<numElements;++i ){
    	printf("%f \n",c[i]);
    }

    // Free device global memory
    cudaFree(a_gpu);
    cudaFree(b_gpu);
    cudaFree(c_gpu);
    
    printf("Done\n");
    return 0;
}
```

In [4]:
!nvcc gpuAdd.cu -o gpu

In [5]:
!./gpu

[Vector addition of 15 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 1 blocks of 256 threads
Copy output data from the CUDA device to the host memory
0.000000 
2.000000 
6.000000 
12.000000 
20.000000 
30.000000 
42.000000 
56.000000 
72.000000 
90.000000 
110.000000 
132.000000 
156.000000 
182.000000 
210.000000 
Done


In [6]:
!nvidia-smi

Mon Sep 14 18:01:19 2015       
+------------------------------------------------------+                       
| NVIDIA-SMI 346.46     Driver Version: 346.46         |                       
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 780     Off  | 0000:01:00.0     N/A |                  N/A |
| 26%   37C    P0    N/A /  N/A |    572MiB /  3071MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 1ra Implementación de PyCUDA

In [7]:
from pycuda import autoinit
from pycuda import gpuarray
import numpy as np

In [8]:
aux = range(15)
a = np.array(aux).astype(np.float32)
b = (a*a).astype(np.float32)
c = np.zeros(len(aux)).astype(np.float32)

In [9]:
a_gpu = gpuarray.to_gpu(a)

In [10]:
b_gpu = gpuarray.to_gpu(b)
c_gpu = gpuarray.to_gpu(c)

In [11]:
aux_gpu = a_gpu+b_gpu

In [12]:
aux_gpu.gpudata

In [13]:
type(aux_gpu)

pycuda.gpuarray.GPUArray

In [14]:
a_gpu,b_gpu,c_gpu

(array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.], dtype=float32),
 array([   0.,    1.,    4.,    9.,   16.,   25.,   36.,   49.,   64.,
          81.,  100.,  121.,  144.,  169.,  196.], dtype=float32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.], dtype=float32))

### 2da Implementación de PyCUDA

In [15]:
from pycuda.elementwise import ElementwiseKernel

In [16]:
c_gpu.dtype

dtype('float32')

In [17]:
myCudaFunc = ElementwiseKernel(arguments = "float *a, float *b, float *c",
                               operation = "c[i] = a[i]+b[i]",
                               name = "mySumK")

In [18]:
myCudaFunc(a_gpu,b_gpu,c_gpu)

In [19]:
c_gpu

array([   0.,    2.,    6.,   12.,   20.,   30.,   42.,   56.,   72.,
         90.,  110.,  132.,  156.,  182.,  210.], dtype=float32)

### 3ra Implementación de PyCUDA

In [20]:
from pycuda.compiler import SourceModule

In [21]:
cudaCode = open("gpuAdd.cu","r")
myCUDACode = cudaCode.read()

In [22]:
myCode = SourceModule(myCUDACode)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(20): warning: linkage specification is not allowed

kernel.cu(20): warning: linkage specification is not allowed


  if __name__ == '__main__':


In [23]:
importedKernel = myCode.get_function("vectorAdd")

In [24]:
!ls

0_Simple	       README.md
addCPU		       README.md~
Animation.ipynb        SchrodingerBOX.pdf
cpua		       Sesion1_Introduccion_Python.ipynb
cpuAdd.c	       Sesion1_Test.ipynb
cpuAdd.c~	       Sesion2_Intento_de_solucion_GPE.ipynb
CUDAkernelsCONS.cu     Sesión3.1_ExploreGPU.ipynb
CUDAkernelsCONS.cu~    Sesion3_CUDA&PyCuda1st.ipynb
CUDAkernels.cu	       Sesion4_PyCUDA_2nd.ipynb
CUDAkernels.cu~        Sesion5_MemoriasCUDA.ipynb
gpu		       Sesion6_PDE_CUDA_670.ipynb
gpuAdd		       Sesion6_PDE_CUDA_780.ipynb
gpuAdd.cu	       Sesion6_PDE_CUDA.ipynb
gpuAdd.cu~	       Sesion6_PDE_CUDA-Tesla.ipynb
imagen		       Sesion7_LibreriasGPU.ipynb
intervalo.py	       Sesion8_PDE_CUDA_Schrodinger.ipynb
intervalo.pyc	       Some_bench.ipynb
libpeerconnection.log  taylor.py
License.txt	       taylor.py~
Longuet-Higgins.ipynb  taylor.pyc
N-Body.ipynb


In [25]:
nData = len(a)

In [26]:
nData

15

In [27]:
nThreadsPerBlock = 256
nBlockPerGrid = 1
nGridsPerBlock = 1

In [28]:
c_gpu.set(c)

In [29]:
c_gpu

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.], dtype=float32)

In [30]:
importedKernel(a_gpu.gpudata,b_gpu.gpudata,c_gpu.gpudata,block=(256,1,1))

In [31]:
c_gpu

array([   0.,    2.,    6.,   12.,   20.,   30.,   42.,   56.,   72.,
         90.,  110.,  132.,  156.,  182.,  210.], dtype=float32)

#Resumen

Desde una perspectiva de cálculo científico podemos resumir a este punto lo siguiente:

**Kernel**
>La funcion elemental de parelización es conocido como *Kernel* de CUDA, se pueden imaginar a estas fucniones escritas con sintaxis de C# como las funciones que ejecutaran en paralelo en cada procesador en los miles que estan dentro de la GPU.

> __global__
> __device__
> __host__

**¿En donde esta la paralelización?**
>Cada vez que se llama a un kernel es necesarion darle una distribucion de hilos (o _threads_) los cuales se organizan en bloques (_blocks_) y estos a su vez en un _grid_ (estos puden poseer distintas dimensiones: 1D,2D,3D) . Estos threads son copias del kernel y cada uno es un proceso a llevarse a cabo en la GPU, es decir si por ejemplo lanzamos un grid con 5 bloques (_gridDim_= (5,1,1)) con 10 threads por bloque (_blockDim_ = (10,1,1)), entonces habremos lanzado 50 tareas en paralelo.
Si bien los kernels a ejecutar por los thread son copias del que escribimos originalmente, la diferenciación se da mediante el asignamiento de un contador a cada proceso, la manera usual de determinar este **indice de proceso global** se ejemplifica asi:
![Alt text](imagen/CUDAmodelThreads.png)

>(**ojo** este pude cambiar dependiendo de la dimension de bloques y de threads)
![Alt text](imagen/cuda-grid.png)

>Para nuestro ejemplo de suma de vectores hemos usado el **indice de proceso global** para que cada thread realice la suma sobre una componente distinta de los vectores. Es en este punto donde aparece la paralelizacion, ya que cada thread sumo una componente distinta del vector.

**PyCUDA**
>Esta biblioteca de python surge como un desarrollo que nos permite en principio hacer todo lo que podemos hacer con CUDA C de una manera más sencilla. Esta biblioteca tiene distintos niveles de implementacion que nos permite desde usar Kernels de CUDA C, hasta no usar ningun kernel explicitamente.
>Una de las virtudes que hemos visto es el uso de la clase **GPUArray** la cual nos permite gestionar de manera sencilla la memoria, asignacion de valores, estructura, transferencias de datos, etc. entre CPU y GPU. Esta clase de pyCUDA mantiene la estructura de arreglos de la biblioteca **numpy** y de manera natural se extienden muchas de las fuciones.

>Una vez inicializado algun contexto de pyCUDA podemos hacer uso de la clase GPUArray, la manera mas simple de generar un arreglo en la memoria global de la GPU es mediante _gpuarray.to_gpu()_ en donde el valor que se pasa a la funcion es un arreglo de **numpy**. Aunque todos los arreglos en memoria global de GPU son arreglos lineales, la clase GPUArray maneja la posibilidad de preservar las dimensiones del arreglo. Desde un Kernel siempre que accedamos a nuestros datos sera de manera lineal (es decir, todos los datos son a lo mas vectores), por lo que el arreglo de arreglos (matrices o mallas 3D) no es posible utilizarlo en memoria global de manera natrural.

### Referencias

#<a href="http://documen.tician.de/pycuda/">pyCUDA</a>

#<a href="http://docs.scipy.org/doc/numpy/reference/">Numpy</a>

#<a href="http://docs.nvidia.com/cuda">CUDA</a>